In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import glob
import soundfile as sf
import librosa
import librosa.display
import cv2
import tensorflow as tf
import keras
import h5py
warnings.filterwarnings('ignore')

In [2]:
from keras.models import load_model
lid_model=load_model('../input/lid-model-and-weights/LID_Model.h5')
print("Model Loaded!!!")

2022-09-16 18:24:47.883391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 18:24:47.982841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 18:24:47.983760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 18:24:47.985505: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model Loaded!!!


In [3]:
lid_model.summary()

Model: "LID_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 196, 196, 64)      128       
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 98, 98, 128)       8320      
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 49, 49, 128)       16512     
_________________________________________________________________
flatten_23 (Flatten)         (None, 307328)            0         
_________________________________________________________________
dense_46 (Dense)             (None, 64)                19

In [4]:
# Loading_weights:
lid_model.load_weights('../input/lid-model-and-weights/LID_Model_weights.h5')
print("Weights are loaded in Model!!")

Weights are loaded in Model!!


In [5]:
def output_duration(length):
    # Pass length as len(data)/samplerate to know length of audio file.
    hours = length // 3600  # calculate in hours
    length %= 3600
    mins = length // 60  # calculate in minutes
    length %= 60
    seconds = length  # calculate in seconds
    #print("Total Duration is: ",hours,":",mins,": ",seconds)
    return seconds

In [6]:
import math
def fix_audio_segment_to_30_seconds(audio_segment,time):
    # Taking sample rate as 22050.
    target_len = 30 * 22050
    if time<30:
        audio_segment = np.concatenate([audio_segment]*((int)(math.ceil(10/time))), axis=0)
    audio_segment = audio_segment[0:target_len]
    return audio_segment

In [7]:
from sklearn.preprocessing import MinMaxScaler
def spectrogram(audio_segment):
    # Compute Mel-scaled spectrogram image
    hl = 196 # image_width
    spec = librosa.feature.melspectrogram(audio_segment,
                                     n_mels=196, 
                                     hop_length=int(hl))

    # Logarithmic amplitudes
    image = librosa.core.power_to_db(spec)

    # Convert to numpy matrix
    image_np = np.asmatrix(spec)
    # For Preprocessing
    image_np = MinMaxScaler().fit_transform(image)
    # Controlling width of image
    res = image_np[:,:196]
    #print(res.shape)
    return res

In [8]:
def process(data,time):
    data_10=fix_audio_segment_to_30_seconds(data,time)
    #print("After Fixing to 30 second :",data_10.shape)
    arr_img=spectrogram(data_10) * 255
    arr_img=arr_img.astype('uint8')
    return arr_img

In [9]:
lid_arr=['Malyalam', 'Tamil', 'Hindi', 'Bengali', 'Kannada', 'Telgu', 'Marathi']

In [10]:
def predict(path):
    flag=False
    data,samplerate=librosa.load(path,res_type='kaiser_fast')
    time=output_duration(int(len(data)/samplerate))
    if time>0:
        flag=True
        X=process(data,time)
        X=np.expand_dims(X,axis=-1)
        X=np.expand_dims(X,axis=0)
        result=lid_model.predict(X)
        result=np.where(result==np.max(result),1,0)
        result=lid_arr[np.argmax(result)]
        return result
    return "Audio File is Empty!!"

In [11]:
!pip install flask-ngrok
!pip install flask-bootstrap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460123 sha256=f78b9c4eec24a542bb37ddbf63e49972082d69617c4a875a1b51666a36dd70b2
  Stored in directory: /root/.cache/pip/wheels/67/a2/d6/50d039c9b59b4caca6d7b53839c8100354a52ab7553d2456eb
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3946 sha256=1ff8659b2d0064a79ce9b1db39a4447db7b00d132795fb82704f96cbdda305eb
  Stored in directory: /root/.cache/pip/wheels/64/34/11/053f47218984c9a31a00f911ed98dda036b867481dcc527a12
Successfully built flask-bootstrap visitor


In [12]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=546630bdd45356d0b086fb6d9d2233d5f6adb028a33d148a61dbd996d5428ff9
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [13]:
! ngrok authtoken 2EqY2bdZrn582Kmu8ltZp7UZoMG_7GiyGqtqzKqGeMTQNW2Ex

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml                                      


In [14]:
pip install gevent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 12.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 24.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
from __future__ import division, print_function
from flask.templating import render_template
from flask import Flask
# coding=utf-8
import sys
import json
import os
import glob
import re
import base64
# Flask utils
from flask import Flask, redirect, url_for, request, render_template,jsonify
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
from flask_ngrok import run_with_ngrok
app = Flask(__name__,template_folder='../input/slidflask/Project_2/templates',static_folder='../input/slidflask/Project_2/static')
run_with_ngrok(app)


@app.route("/")
def home():
    return render_template("index.html")


@app.route('/predict', methods=['POST'])
def upload():
    if request.method == 'POST':
        #print("Got a Request!!")
        json_data=request.get_json();
        data_audio=json_data['file_1']
        #print("Data Fetched!!!")
        data_audio=data_audio[data_audio.index(","):]
        decoded_data_audio=base64.b64decode((data_audio))
        #print("Data Decoded!!")
        audio_file = open('./audio_sample.wav', 'wb')
        audio_file.write(decoded_data_audio)
        #print("Data Written in File")
        audio_file.close()
        #print("File Saved!!")
        result=predict('./audio_sample.wav')
        #print("File encoding done!!")
        return jsonify({'b64':result})
    return None

In [16]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on http://d71b-34-82-246-244.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Got a Request!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!


2022-09-16 18:27:10.289313: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-16 18:27:11.392744: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


File encoding done!!
File encoding done!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
File encoding done!!
File encoding done!!
Got a Request!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Fetched!!!
Data Decoded!!
Data Written in File
Data Written in File
File Saved!!
File Saved!!
File encoding done!!
File encoding done!!
Got a Request!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
File encoding done!!
File encoding done!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
File encoding done!!
File encoding done!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written in File
File Saved!!
Got a Request!!
Data Fetched!!!
Data Decoded!!
Data Written i